In [1]:
import os
pdf_path = "Documents/5C-LYNX_Chatbot-191124-0622.pdf"
print(pdf_path)

Documents/5C-LYNX_Chatbot-191124-0622.pdf


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
 

True

In [3]:
# Requires !pip install PyMuPDF, see: https://github.com/pymupdf/pymupdf
import fitz # (pymupdf, found this is better than pypdf for our use case, note: licence is AGPL-3.0, keep that in mind if you want to use any code commercially)
from tqdm.auto import tqdm # for progress bars, requires !pip install tqdm 

def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip() # note: this might be different for each doc (best to experiment)

    # Other potential text formatting functions can go here
    return cleaned_text

# Open PDF and get lines/pages
# Note: this only focuses on text, rather than images/figures etc
def open_and_read_pdf(pdf_path: str) -> list[dict]:
    """
    Opens a PDF file, reads its text content page by page, and collects statistics.

    Parameters:
        pdf_path (str): The file path to the PDF document to be opened and read.

    Returns:
        list[dict]: A list of dictionaries, each containing the page number
        (adjusted), character count, word count, sentence count, token count, and the extracted text
        for each page.
    """
    doc = fitz.open(pdf_path)  # open a document
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):  # iterate the document pages
        # if page_number <= 2:  # Skip the first 2 pages
        #  continue
        text = page.get_text()  # get plain text encoded as UTF-8
        text = text_formatter(text)
        pages_and_texts.append({"page_number": page_number - 0,  # adjust page numbers since our PDF starts on page 5
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text) / 4,  # 1 token = ~4 chars, see: https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them
                                "text": text})
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts


/Users/saiteja.kola/Downloads/Work/Sources/LLM_CHAT/LLM_CHAT/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
1it [00:00, 116.90it/s]


[{'page_number': 0,
  'page_char_count': 3560,
  'page_word_count': 640,
  'page_sentence_count_raw': 36,
  'page_token_count': 890.0,
  'text': '1.   2.   3.   1.   2.   a.   b.   3.   4.   a.   LYNX_Chatbot User Creation in TTI: User creation is managed by company fleet managers. Navigate to company management and select action button and add user. for adding user fleet manager needs to select the company for which user needs to be added and than the required data asked in the  stepper form to create the user. Once user is created by fleet manager created user will get the email from OKTA where they get the link to activate the account. Once  user activated the account. They can use the credentials to login to the lynx fleet application. Password reset: User can reset the password from the login screen of the lynx fleet application. Company tiering: feature users have access to depends on company tiering level. we have   tiering available for each company. access, core and enhance Fe

In [5]:
import random
random.sample(pages_and_texts, k=1)

[{'page_number': 0,
  'page_char_count': 3560,
  'page_word_count': 640,
  'page_sentence_count_raw': 36,
  'page_token_count': 890.0,
  'text': '1.   2.   3.   1.   2.   a.   b.   3.   4.   a.   LYNX_Chatbot User Creation in TTI: User creation is managed by company fleet managers. Navigate to company management and select action button and add user. for adding user fleet manager needs to select the company for which user needs to be added and than the required data asked in the  stepper form to create the user. Once user is created by fleet manager created user will get the email from OKTA where they get the link to activate the account. Once  user activated the account. They can use the credentials to login to the lynx fleet application. Password reset: User can reset the password from the login screen of the lynx fleet application. Company tiering: feature users have access to depends on company tiering level. we have   tiering available for each company. access, core and enhance Fe

In [7]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,0,3560,640,36,890.0,1. 2. 3. 1. 2. a. b. 3. 4. a...


In [12]:
# Get stats
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,1.0,1.0,1.0,1.0,1.0
mean,0.0,3560.0,640.0,36.0,890.0
std,NaN,NaN,NaN,NaN,NaN
min,0.0,3560.0,640.0,36.0,890.0
25%,0.0,3560.0,640.0,36.0,890.0
50%,0.0,3560.0,640.0,36.0,890.0
75%,0.0,3560.0,640.0,36.0,890.0
max,0.0,3560.0,640.0,36.0,890.0


In [13]:
from spacy.lang.en import English # see https://spacy.io/usage for install instructions

nlp = English()

# Add a sentencizer pipeline, see https://spacy.io/api/sentencizer/ 
nlp.add_pipe("sentencizer")

# Create a document instance as an example
doc = nlp("This is a sentence. This another sentence.")
assert len(list(doc.sents)) == 2

# Access the sentences of the document
list(doc.sents)

[This is a sentence., This another sentence.]

In [14]:
for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)
    
    # Make sure all sentences are strings
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]
    
    # Count the sentences 
    item["page_sentence_count_spacy"] = len(item["sentences"])

100%|██████████| 1/1 [00:00<00:00, 88.14it/s]


In [15]:

df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy
count,1.0,1.0,1.0,1.0,1.0,1.0
mean,0.0,3560.0,640.0,36.0,890.0,33.0
std,NaN,NaN,NaN,NaN,NaN,NaN
min,0.0,3560.0,640.0,36.0,890.0,33.0
25%,0.0,3560.0,640.0,36.0,890.0,33.0
50%,0.0,3560.0,640.0,36.0,890.0,33.0
75%,0.0,3560.0,640.0,36.0,890.0,33.0
max,0.0,3560.0,640.0,36.0,890.0,33.0


In [16]:
# Define split size to turn groups of sentences into chunks
num_sentence_chunk_size = 18

# Create a function that recursively splits a list into desired sizes
def split_list(input_list: list, 
               slice_size: int) -> list[list[str]]:
    """
    Splits the input_list into sublists of size slice_size (or as close as possible).

    For example, a list of 17 sentences would be split into two lists of [[10], [7]]
    """
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

# Loop through pages and texts and split sentences into chunks
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

100%|██████████| 1/1 [00:00<00:00, 17476.27it/s]


In [17]:
# Sample an example from the group (note: many samples have only 1 chunk as they have <=10 sentences total)
random.sample(pages_and_texts, k=1)

[{'page_number': 0,
  'page_char_count': 3560,
  'page_word_count': 640,
  'page_sentence_count_raw': 36,
  'page_token_count': 890.0,
  'text': '1.   2.   3.   1.   2.   a.   b.   3.   4.   a.   LYNX_Chatbot User Creation in TTI: User creation is managed by company fleet managers. Navigate to company management and select action button and add user. for adding user fleet manager needs to select the company for which user needs to be added and than the required data asked in the  stepper form to create the user. Once user is created by fleet manager created user will get the email from OKTA where they get the link to activate the account. Once  user activated the account. They can use the credentials to login to the lynx fleet application. Password reset: User can reset the password from the login screen of the lynx fleet application. Company tiering: feature users have access to depends on company tiering level. we have   tiering available for each company. access, core and enhance Fe

In [18]:
# Create a DataFrame to get stats
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy,num_chunks
count,1.0,1.0,1.0,1.0,1.0,1.0,1.0
mean,0.0,3560.0,640.0,36.0,890.0,33.0,2.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.0,3560.0,640.0,36.0,890.0,33.0,2.0
25%,0.0,3560.0,640.0,36.0,890.0,33.0,2.0
50%,0.0,3560.0,640.0,36.0,890.0,33.0,2.0
75%,0.0,3560.0,640.0,36.0,890.0,33.0,2.0
max,0.0,3560.0,640.0,36.0,890.0,33.0,2.0


In [19]:
import re

# Split each chunk into its own item
pages_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]
        
        # Join the sentences together into a paragraph-like structure, aka a chunk (so they are a single string)
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" -> ". A" for any full-stop/capital letter combo 
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        # Get stats about the chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 characters
        
        pages_and_chunks.append(chunk_dict)

# How many chunks do we have?
len(pages_and_chunks)

100%|██████████| 1/1 [00:00<00:00, 2369.66it/s]


2

In [20]:
# View a random sample
random.sample(pages_and_chunks, k=1)

[{'page_number': 0,
  'sentence_chunk': '24 hours 48 hours 7 days A , where the user can select a specific date and time, down to the second. Note that the custom period is limited to 7 custom time range days due to the large data volume. Need of beacon Sensors: fleet manager would like to know which truck/tractor is near my trailer to ensure the trailer is on the correct route. Sometimes, the situation occurs where the trailer gets hitched to the wrong truck and ends up going to the wrong destination. This results in our customers experiencing lost time and revenue. :Process to commission beacon sensors The beacons will be installed on the back of trucks The beacons will be commissioned in Lynx Fleet and associated with a truck. The Carrier team member will enter the beacon sensor MAC ID, truck license plate number, and truck description during commissioning. Beacon sensors can be commissioned individually. Beacon sensors can be commissioned via bulk upload When a company opts into be

In [21]:
# Get stats about our chunks
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,2.0,2.00,2.00,2.00
mean,0.0,1761.00,301.50,440.25
std,0.0,123.04,14.85,30.76
min,0.0,1674.00,291.00,418.50
25%,0.0,1717.50,296.25,429.38
50%,0.0,1761.00,301.50,440.25
75%,0.0,1804.50,306.75,451.12
max,0.0,1848.00,312.00,462.00


In [22]:

pages_and_chunks_over_min_token_len = df.to_dict(orient="records")
# pages_and_chunks_over_min_token_len[:2]
print(pages_and_chunks_over_min_token_len)

[{'page_number': 0, 'sentence_chunk': '1. 2. 3. 1. 2. a.  b.  3. 4. a.  LYNX_Chatbot User Creation in TTI: User creation is managed by company fleet managers. Navigate to company management and select action button and add user.for adding user fleet manager needs to select the company for which user needs to be added and than the required data asked in the stepper form to create the user. Once user is created by fleet manager created user will get the email from OKTA where they get the link to activate the account. Once user activated the account. They can use the credentials to login to the lynx fleet application. Password reset: User can reset the password from the login screen of the lynx fleet application. Company tiering: feature users have access to depends on company tiering level.we have  tiering available for each company.access, core and enhance Feature Access Details: API Portal: Available in Access, Core, Enhance Live Map View: Available in Access, Core Live Map View - Geof

In [31]:
# # Requires !pip install sentence-transformers
# from sentence_transformers import SentenceTransformer
# embedding_model = SentenceTransformer(model_name_or_path="dunzhang/stella_en_1.5B_v5", 
#                                       device="mps") # choose the device to load the model to (note: GPU will often be *much* faster than CPU)

from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings ## embeddings using Interface endpoint

embeddings = HuggingFaceInferenceAPIEmbeddings(api_key= os.environ['HUGGING_API_KEY'], model_name="sentence-transformers/all-mpnet-base-v2")
# embeddings = HuggingFaceEndpointEmbeddings() ## embedding using local huggingface
print(embeddings)


api_key=SecretStr('**********') model_name='sentence-transformers/all-mpnet-base-v2' api_url=None additional_headers={}


In [24]:
# Create a list of sentences to turn into numbers
sentences = [
    "The Sentences Transformers library provides an easy and open-source way to create embeddings.",
    "Sentences can be embedded one by one or as a list of strings.",
    "Embeddings are one of the most powerful concepts in machine learning!",
    "Learn to use embeddings well and you'll be well on your way to being an AI engineer."
]

# Sentences are encoded/embedded by calling model.encode()
embeddings =  embeddings.embed_documents(sentences)
embeddings_dict = dict(zip(sentences, embeddings))
print(embeddings_dict)
# See the embeddings
for sentence, embedding in embeddings_dict.items():
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

{'The Sentences Transformers library provides an easy and open-source way to create embeddings.': [-0.020798319950699806, 0.030316464602947235, -0.020121799781918526, 0.06864849478006363, -0.02552560716867447, -0.008476873859763145, -0.00020723622583318502, -0.0632377415895462, 0.0281606987118721, -0.033335376530885696, 0.03026341088116169, 0.05307215824723244, -0.05035270005464554, 0.026228871196508408, 0.03333137184381485, -0.045157741755247116, 0.036304496228694916, -0.0013711730716750026, -0.012017124332487583, 0.0114947110414505, 0.05045110359787941, 0.047085680067539215, 0.021191375330090523, 0.05146066099405289, -0.020374629646539688, -0.03588895872235298, -0.0006677835481241345, -0.02943938970565796, 0.04958592355251312, -0.010563945397734642, -0.015201376751065254, -0.0013175965286791325, 0.044819723814725876, 0.015602342784404755, 8.603794299233414e-07, -0.0012139284517616034, -0.023797864094376564, -0.0009093867265619338, 0.007344875484704971, -0.0025393629912286997, 0.05233

In [26]:
# Turn text chunks into a single list
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]
print(text_chunks)

['1. 2. 3. 1. 2. a.  b.  3. 4. a.  LYNX_Chatbot User Creation in TTI: User creation is managed by company fleet managers. Navigate to company management and select action button and add user.for adding user fleet manager needs to select the company for which user needs to be added and than the required data asked in the stepper form to create the user. Once user is created by fleet manager created user will get the email from OKTA where they get the link to activate the account. Once user activated the account. They can use the credentials to login to the lynx fleet application. Password reset: User can reset the password from the login screen of the lynx fleet application. Company tiering: feature users have access to depends on company tiering level.we have  tiering available for each company.access, core and enhance Feature Access Details: API Portal: Available in Access, Core, Enhance Live Map View: Available in Access, Core Live Map View - Geofence: Available in Access, Core Live 

In [27]:
#Vector Search DB In PineCode
##Create a pinecone index first and the run this code
import pinecone
from pinecone import Pinecone
pc = Pinecone(api_key = os.environ['PINECONE_API_KEY'])
index = pc.Index("llmchat")
print(index)
index_name = 'llmchat'

In [28]:
from langchain.schema import Document
document_list = [
    Document(
        page_content=text_chunks["sentence_chunk"],
        metadata={"page": text_chunks["page_number"]}
    )
    for text_chunks in pages_and_chunks_over_min_token_len
]
print(document_list)

[Document(metadata={'page': 0}, page_content='1. 2. 3. 1. 2. a.  b.  3. 4. a.  LYNX_Chatbot User Creation in TTI: User creation is managed by company fleet managers. Navigate to company management and select action button and add user.for adding user fleet manager needs to select the company for which user needs to be added and than the required data asked in the stepper form to create the user. Once user is created by fleet manager created user will get the email from OKTA where they get the link to activate the account. Once user activated the account. They can use the credentials to login to the lynx fleet application. Password reset: User can reset the password from the login screen of the lynx fleet application. Company tiering: feature users have access to depends on company tiering level.we have  tiering available for each company.access, core and enhance Feature Access Details: API Portal: Available in Access, Core, Enhance Live Map View: Available in Access, Core Live Map View

In [30]:
# converting the document into embeddings and insertion into pinecone vector db based on the chunks - Each chunk is stored as a single record
from langchain_pinecone import PineconeVectorStore
vectorstore_from_docs = PineconeVectorStore.from_documents(
        document_list,
        index_name=index_name,
        embedding=embeddings
    )